In [1]:
!pip3 install kfp


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 119 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.5 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-0.5.1-py3-none-any.whl size=163151 sha256=da5b540ae9834d37659146f0576997ffd8f7a7e2b305e1eb7b2a99dd4745930b
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/26/f9/e3836cb6e6cabd63ef912304e18a852ac29cb870a4a0b85f98
  Created wheel for kfp-server-api: filename=kfp_server_api-0.5.0-py3-none-any.whl size=106319 sha256=84f55948cc254c0f836dffdfd51574a828ae8a503a2ca9198acf7a27ca2aaea7
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/36/4e/bfe2efeeea4f74f04984ebe1d44136202b72191302f4760951
  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=24671 sha256=3bcfd573a91f5f4c46d23509ac3fee9a0cf351b414e00ed505a8f71d0e6a1141
  Stored in di

In [1]:
import kfp
import kubernetes

In [2]:

container_manifest = {
    "apiVersion": "sparkoperator.k8s.io/v1beta2",
    "kind": "SparkApplication",
    "metadata": {
        "name": "spark-app",
        "namespace": "kubeflow"
    },
    "spec": {
        "type": "Scala",
        "mode": "cluster",
        "image": "mavencode/covid-basis-vectors:1.0.0-charles",
        "imagePullPolicy": "Always",
        "mainClass": "org.kftraining.covid.App",
        "mainApplicationFile": "local:///covid-0.1-jar-with-dependencies.jar", # See the Dockerfile
        "arguments": ["245", "15", "1"],
        "sparkVersion": "2.4.5",
        "restartPolicy": {
            "type": "Never"
        },
        "driver": {
            "cores": 1,
            "coreLimit": "1200m",
            "memory": "512m",
            "labels": {
                "version": "2.4.5",
            },
            "serviceAccount": "spark-operatoroperator-sa", # also try spark-operatoroperator-sa
        },
        "executor": {
            "cores": 1,
            "instances": 1,
            "memory": "4084m"
        },
        "labels": {
            "version": "2.4.5"
        },

    }
}

In [16]:
@kfp.dsl.pipeline(
    name="Covid DICOM Pipe v2",
    description="Create Basis Vectors for Lung Images"
)
def covid_dicom_pipeline():
    vop = kfp.dsl.VolumeOp(
        name="requisition-PVC",
        resource_name="datapvc",
        size="20Gi", #10 Gi blows up...
        modes=kfp.dsl.VOLUME_MODE_RWO
    )
    step1 = kfp.dsl.ContainerOp(
        name="download-dicom",
        image="mavencodev/download-dicom:1.0.13-charles",
        command=["/run.sh"],
        pvolumes={"/data": vop.volume}
    )
    step2 = kfp.dsl.ContainerOp(
        name="convert-dicoms-to-vectors",
        image="mavencodev/covid-prep-dicom:1.0.7-charles",
        arguments=[
            '--output_dir', "/mnt/data",
        ],
        command=["python", "/program.py"],
        pvolumes={"/mnt/data": step1.pvolume}
    )

    #     rop = kfp.dsl.ResourceOp(
    #         name="calculate-basis-vectors",
    #         k8s_resource=container_manifest,
    #         action="create",
    #         success_condition="status.applicationState.state == COMPLETED"
    #     ).after(step2)
    #
    # pyviz = kfp.dsl.ContainerOp(
    #     name="visualize-slice-of-dicom",
    #     image="mavencodev/visualize-dicom-output:1.0.0-charles",
    #     command=["python", "/program.py"],
    #     arguments=[ '--bucket_name', "covid-dicoms",],
    # ).after(rop)

kfp.compiler.Compiler().compile(covid_dicom_pipeline,"dicom-pipeline-2.zip")
client = kfp.Client(host='pipelines-api.kubeflow.svc.cluster.local:8888')
# client.list_pipelines()
pipeline_info = client.upload_pipeline('dicom-pipeline-2.zip',pipeline_name='dicom-pipeline-2')

In [17]:

my_experiment = client.create_experiment(name='dicom-experiments')
my_run = client.run_pipeline(my_experiment.id, 'my-run1', 'dicom-pipeline-2.zip')